<a href="https://colab.research.google.com/github/Amarnath-188/Design-Optimization/blob/Project-1/Thermodynamic_parameter_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Introduction**



The Wilson activity coefficient model is a semi-empirical model used to describe the non-ideal behavior of liquid mixtures. It is based on the assumption that the excess Gibbs free energy of mixing is due to the different sizes and interactions of the molecules in the mixture. The Wilson model has been shown to be accurate for a wide range of mixtures, including ethanol-cyclohexane.

# **Ebulliometer**


An ebulliometer is a laboratory instrument used to measure the boiling point of a liquid. It works by heating a liquid until it boils, and then measuring the temperature of the vapor. The boiling point of a liquid is related to its vapor pressure, which is a measure of the tendency of the liquid to evaporate.

#**Vapor Liquid Equilibrium (VLE)**

VLE is a state of equilibrium in which a liquid and its vapor are present at the same temperature and pressure.

#**Determining VLE Parameters from Ebulliometer Data**

The following steps can be used to determine the VLE parameters for an ethanol-cyclohexane mixture from ebulliometer data:



1.   Measure the boiling point of a series of ethanol-cyclohexane mixtures with different compositions.
2.   Use the measured boiling points to calculate the vapor pressures of the mixtures.
3. Use the Wilson activity coefficient model to relate the vapor pressures to the mole fractions of ethanol and cyclohexane in the liquid phase.
4. Fit the Wilson activity coefficient model to the experimental data to determine the values of the model parameters.


#**Azeotrope**

An azeotrope is a mixture of two or more liquids that has a constant boiling point over a range of compositions. In other words, the composition of the vapor phase is the same as the composition of the liquid phase. Azeotropes are important in industrial processes, such as distillation.

**Activity Coefficients at Infinitely Dilute Compositions**

The activity coefficient of a component at its infinitely dilute composition is defined as the ratio of the fugacity of the component in the mixture to the fugacity of the pure component at the same temperature and pressure. The fugacity is a measure of the escaping tendency of a component from a mixture.

The liquid and vapor compositions of this binary mixture are related by the following thermodynamic relationships

$y_1 .P = x_1 \gamma_1 P_1^{sat}$


$y_2 .P = x_2 \gamma_2 P_2^{sat}$

In the context of the Wilson equation, y1 represents the vapor mole fraction, P denotes the pressure, x1 is the liquid mole fraction, gamma1 represents the activity coefficient, which deviates from 1.0 in non-ideal mixtures, and P1sat is the vapor pressure of the pure component. This equation is equally applicable to component 2 in the mixture, with a corresponding equation denoted by subscript 2.

The Wilson equation is employed to forecast the activity coefficients, gamma1 and gamma2, across various liquid compositions.




$\Lambda_{12}= \frac{V2}{V1} e^{\frac{-\lambda_{12}}{R_g T}}$

$\Lambda_{21}= \frac{V1}{V2} e^{\frac{-\lambda_{21}}{R_g T}}$

$\ln(\gamma_1) = -\ln(x_1 + \Lambda_{12} x_2) + x_2(\frac{\Lambda_{12}}{x_1 + \Lambda_{12} x_2}-\frac{\Lambda_{21}}{x_2 + \Lambda_{21} x_1})$

$\ln(\gamma_2) = -\ln(x_2 + \Lambda_{21} x_1) - x_1(\frac{\Lambda_{12}}{x_1 + \Lambda_{12} x_2}-\frac{\Lambda_{21}}{x_2 + \Lambda_{21} x_1})$


Correlations for the vapor pressure $P_1^{sat}$ and density (rho) of numerous commonly encountered pure components can be found in BYU's DIPPR database. In this situation, $P_1^{sat}$ varies with temperature in accordance with a specified function.

$P_{sat} = e^{(A_o + \frac{B_o}{T}+ C_o \ln(T)+D_o T^{E_o})}$

$\rho = \frac{A}{(1+1-{\frac{T}{C}})}$

$v = 1/\rho$

$\rho$ and $v$ is also a function of Temperature

The formula for determining the degrees of freedom (DOF) in a system involving multiple components and phases is expressed as DOF = 2 + #Components - #Phases. Here, the system consists of two phases (liquid and vapor) and two components (ethanol and cyclohexane), resulting in two degrees of freedom that require specification. In this scenario, a choice must be made to constrain two out of the four measured values for the system, involving x1, y1, P, or T. It is advisable to constrain the values of x1 and P, as demonstrated in the tutorial below.

In [1]:
pip install gekko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 85.9 MB/s eta 0:00:00


In [2]:
# importing libraries
from gekko import GEKKO
from scipy.optimize import fsolve
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
#Molar gas constant
R = 1.9858775  # cal/mol K

def vaporP(T: float, compound: list):
    '''
    T              ---> temp K
    param compound ---> vapor pressure
    return         ---> Vapor Pressure in Pa
    '''
    VP = np.exp(compound[0] + compound[1]/T + compound[2]*np.log(T) + compound[3]*T**compound[4])
    return VP

In [19]:
def liquidMolarVolume(T: float, mvolume: list):
    '''
    T               ---> temp in K
    mvolume         ---> component liquid molar volume
    return          ---> liquid molar volume
    '''
    L_mol_volume = 1 / mvolume[0] * mvolume[1] ** (1 + (1 - T / mvolume[2]) ** mvolume[3])
    return L_mol_volume

In [25]:
def Lij(aij: float, T: float, Vlvali: float,\
            Vlvalj: float):
    '''
    # Vl1 = liquidMolarVolume(T, Vlvali)
    # Vl2 = liquidMolarVolume(T, Vlvalj)
    '''
    Vl1 = Vlvali
    Vl2 = Vlvalj
    V = Vl2 / Vl1 * np.exp(-aij / (R * T))
    return V

In [28]:
def Gamma(a12: float, a21: float, x1: float,\
          Temp: float, Compound: int):


  '''
  Compound 1: Ethanol
  Compound 2: Cyclohexane
  Liquid molar volume
  yi * P =  γi * xi * Pisat
  '''

  Vethanol_mvol = 0.058492
  VCyclochexane_mvol = 0.10882

  x2 = 1.0 - x1
  L12 = Lij(a12, Temp, Vethanol_mvol, VCyclochexane_mvol)
  L21 = Lij(a21, Temp,  VCyclochexane_mvol, Vethanol_mvol)
  L11 = 1.0
  L22 = L11

  if Compound == 1:
      A = (x2 * L12)+x1
      B = x1 * L11 / (x1 * L11 + x2 * L12) \
           +x2 * L21 / (x1 * L21 + x2 * L22)

  elif Compound == 2:

      A = (x1 * L21)+x2
      B = x2 * L22 / (x2 * L22 + x1 * L21) \
         +x1 * L12 / (x2 * L12 + x1 * L11)

  C = np.exp(1.0-np.log(A)-B)

  return C